In [1]:
# GET PARAMETERS
import os 
import sys
import pandas as pd
import torch 
import pickle
import numpy as np 
import importlib
import pickle

# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)


# Personnal imports: 
from constants.paths import SAVE_DIRECTORY, FOLDER_PATH
from examples.accuracy_comparison import load_trainer_ds_from_saved_trial

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No module xgboost
Training and Hyper-parameter tuning with Ray is not possible
Training and Hyper-parameter tuning with Ray is not possible



## Load trained model, without the use of NetMob: 

In [2]:
subfolder = 'K_fold_validation/training_wo_HP_tuning/optim/subway_in_STGCN'
path_model_args = f"{SAVE_DIRECTORY}/{subfolder}/best_models"
model_args = pickle.load(open(f"{path_model_args}/model_args.pkl", 'rb'))


# trial_id1 = 'subway_in_calendar_emb64_out64_Huber_MinMax_bis1'   
# trial_id1 = 'subway_in_calendar_emb64_out64_Huber_MinMax_horizon1_bis1'
# trial_id1 = 'subway_in_calendar_emb64_out64_Huber_MinMax_horizon2_bis1'
# trial_id1 = 'subway_in_calendar_emb64_out64_Huber_MinMax_horizon3_bis1'
# trial_id1 = 'subway_in_calendar_emb64_out64_Huber_MinMax_horizon4_bis1'

# Load Model from trial : 
trial_id = 'subway_in_calendar_emb64_out64_Huber_MinMax_horizon1_bis1'

trial_id_updated = f"_{trial_id}_f5"
args = model_args['model'][trial_id_updated]['args']
model_save_path = f"{path_model_args}/{trial_id_updated}.pkl"
trained_trainer, trained_ds, trained_args = load_trainer_ds_from_saved_trial(args,model_save_path,modification = {('device'): torch.device('cuda:0')})

----------------------------------------
Loading the Complete Dataset for K-fold splitting
Coverage Period: 7392 elts between 2019-03-16 00:00:00 and 2019-05-31 23:45:00
Invalid dates within this fold: 776

>>>Tackle Target dataset: subway_in
   Load data from: //home/rrochas/../../data/rrochas/prediction_validation/subway_in/subway_in.csv
   Init Dataset: 'torch.Size([7392, 40]). 0 Nan values
   TRAIN contextual_ds: torch.Size([2821, 40, 7])
   VALID contextual_ds: torch.Size([940, 40, 7])
   TEST contextual_ds: torch.Size([940, 40, 7])

>>>Loading calendar embedding inputs ...
    dict_keys(['dayofweek_OHE', 'hour_OHE'])
   args_embedding.variable_selection_model_name:  MLP
Init U/Utarget size: torch.Size([4702, 40, 7])/torch.Size([4702, 40, 1]) Train/Valid/Test 2821 940 940

 ===== ERROR WITH prefetch_factor====  
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing

----------------------------------------
Loadi

## Load a New Trainer, with NetMob Inputs : 

In [3]:
from examples.train_model import load_init_model_trainer_ds
from examples.benchmark import local_get_args
from examples.reproductibility.config_STGCN_Subway_in_NetMob_calendar_h1 import modifications

modification = modifications['HPO_subway_in_calendar_embedding_netmob_POIs_STGCN_HuberLossLoss_2025_08_11_23_22_40073_h1']
save_folder = None

args_init = local_get_args(modification['model_name'],
                args_init = None,
                dataset_names=modification['dataset_names'],
                dataset_for_coverage=modification['dataset_for_coverage'],
                modification = modification)
fold_to_evaluate=[args_init.K_fold-1]

trainer,ds,model,args = load_init_model_trainer_ds(fold_to_evaluate,save_folder,args_init,modification,trial_id)

----------------------------------------
Loading the Complete Dataset for K-fold splitting
Coverage Period: 7392 elts between 2019-03-16 00:00:00 and 2019-05-31 23:45:00
Invalid dates within this fold: 776

>>>Tackle Target dataset: subway_in
   Load data from: //home/rrochas/../../data/rrochas/prediction_validation/subway_in/subway_in.csv
   Init Dataset: 'torch.Size([7392, 40]). 0 Nan values
   TRAIN contextual_ds: torch.Size([2821, 40, 7])
   VALID contextual_ds: torch.Size([940, 40, 7])
   TEST contextual_ds: torch.Size([940, 40, 7])

>>>Loading calendar embedding inputs ...
    dict_keys(['dayofweek_OHE', 'hour_OHE'])
   args_embedding.variable_selection_model_name:  MLP

>>>Tackle Contextual dataset:  netmob_POIs
    ATTENTION: Dimension reduction by clustering is applied on the entire dataset. This should be done only on the training set.
    Netmob_T.size(): torch.Size([7392, 556]). Dimensionality reduced by 35.3%


Restraining all datasets to train common dates: 2821 dates
Res

## Transfer Learning: 

In [4]:
# Option 1: Transférer tous les poids possibles et les geler
print("--- Transferring all possible weights ---")
trainer.transfer_weights_from(trained_trainer, freeze_transferred=True, List_not_freezing = ['core_model.output'])

--- Transferring all possible weights ---
62 layers have been frozen.

--- Transfer Summary ---
Kept 41 layers to train.
Found 1 layers with mismatched shapes (weights not transferred):
  - core_model.output.fc1.weight: Dest: torch.Size([64, 192]) vs Source: torch.Size([64, 128])
------------------------

Model structure after transfer:

- Blue:  All parameters in the module were transferred AND are frozen
 - Green:  All parameters in the module were transferred AND are NOT frozen
 - Yellow: Some, but not all, parameters were transferred
 - White: No parameters were transferred

(spatial_attn_per_station): ModuleDict()
(spatial_attn_poi): ModuleDict(
spatial_attn_poi.(netmob_POIs): model(
spatial_attn_poi.netmob_POIs.(mha_list): ModuleList(
spatial_attn_poi.netmob_POIs.mha_list.(0): MHA_layer(
spatial_attn_poi.netmob_POIs.mha_list.0.(mha): MultiHeadAttention(
spatial_attn_poi.netmob_POIs.mha_list.0.mha.(softmax): Softmax(dim=-1)
spatial_attn_poi.netmob_POIs.mha_list.0.mha.(dropout): Dr

In [ ]:
# Option 2: Transférer uniquement les modules 'te' et 'core_model' et les geler
print("\n\n--- Transferring specific modules ('te' and 'core_model') ---")
modules_to_transfer = ['te', 'core_model']
trainer.transfer_weights_from(
    trained_trainer, 
    modules_to_transfer=modules_to_transfer, 
    freeze_transferred=True
)